# 6.4 Exercise

Using the Italian's space agency [SED builder](https://tools.ssdc.asi.it/SED/), build the SED of the source 1ES1215+303. Download the data and use the function in this notebook to convert the file into an `astropy.table`. Plot the SED with the fluxes and the errors (`plt.errorbar`). Use the information you can find online on the source (e.g. redshift, ...) and `agnpy` to make a tentative SSC model that describes the data.

In [ ]:
import numpy as np
import astropy.units as u
from astropy.table import Table
from astropy.io import fits
import matplotlib.pyplot as plt

In [ ]:
def convert_asdc_sed_to_table(filename):
    """Convert a .txt file containing the ASDC SED builder format to an astropy table with the agnpy format"""
    with open(filename) as f:
        contents = f.readlines()

        # substring identifying the line where a new instrument
        # dataset is starting
        substring = " (id ="

        nu = []
        nuFnu = []
        nuFnu_err = []
        instrument = []

        # first four rows are coordinates, redshift and H density
        for content in contents[4:]:
            if substring in content:
                name = content.split(substring)[0].strip("# ")
            if (
                not content.startswith("#")
                and substring not in content
                and "UPPER LIMIT" not in content
            ):
                values = [float(_) for _ in content.split()]
                nu.append(values[0])
                nuFnu.append(values[2])
                nuFnu_err.append(values[3])
                # there might be repeated measurements by the same instruments
                # or a LC?
                # so append the MJD START and STOP to the name
                if values[4] > 0 and values[5] > 0:
                    instrument.append(name + f" MJD_START_{values[4]}_STOP_{values[5]}")
                else:
                    instrument.append(name)

    # convert units
    nu *= u.Hz
    nuFnu *= u.Unit("erg cm-2 s-1")
    nuFnu_err *= u.Unit("erg cm-2 s-1")

    e_ref = nu.to("eV", equivalencies=u.spectral())

    tab = Table(
        [e_ref, nuFnu, nuFnu_err, nuFnu_err, instrument],
        names=["e_ref", "e2dnde", "e2dnde_errn", "e2dnde_errp", "instrument"],
    )
    return tab